# 🏗️ Proyecto Integrador 2: Plataforma Self-Service con GenAI

**Objetivo**: construir una plataforma que permita a usuarios no técnicos generar pipelines ETL, documentación, y reportes usando IA generativa.

## Alcance del Proyecto

- **Duración**: 6-8 horas
- **Dificultad**: Muy Alta
- **Stack**: OpenAI, LangGraph, Airflow, Great Expectations, Streamlit

## Funcionalidades

1. ✅ Generación de pipelines ETL desde descripción en lenguaje natural
2. ✅ Validación automática del código generado
3. ✅ Creación de tests unitarios
4. ✅ Generación de documentación técnica
5. ✅ Dashboard de monitoreo
6. ✅ Sistema de aprobación y deployment

## Parte 1: Arquitectura del sistema

```
User Input (NL)
    ↓
[1. Parser] → extrae requisitos
    ↓
[2. Generator] → genera código ETL
    ↓
[3. Validator] → valida sintaxis, seguridad
    ↓
[4. Tester] → genera y ejecuta tests
    ↓
[5. Documenter] → crea README
    ↓
[6. Reviewer] → revisión humana
    ↓
[7. Deployer] → despliega a producción
```

## Parte 2: Setup

In [ ]:
# pip install openai langgraph streamlit pytest great-expectations
import os
import json
import ast
from openai import OpenAI
from typing import TypedDict, Annotated
import operator

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print('✅ Setup completo')

## Parte 3: Parser de requisitos

In [ ]:
def parse_requirements(user_input: str) -> dict:
    """Extrae requisitos estructurados desde lenguaje natural."""
    prompt = f'''
Extrae los requisitos de este pipeline ETL en formato JSON:

Input del usuario:
{user_input}

Devuelve JSON con:
{{
  "pipeline_name": "nombre_descriptivo",
  "source": {{"type": "csv/api/db", "details": "..."}},
  "transformations": ["lista de transformaciones"],
  "destination": {{"type": "csv/db/s3", "details": "..."}},
  "schedule": "daily/hourly/manual",
  "validations": ["reglas de calidad"]
}}
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.1
    )
    
    return json.loads(resp.choices[0].message.content)

# Test
user_request = '''
Necesito un pipeline que:
1. Lea ventas.csv de S3 bucket "raw-data"
2. Filtre solo ventas de los últimos 30 días
3. Agregue una columna "mes" (YYYY-MM)
4. Calcule total por mes y categoría
5. Escriba a PostgreSQL tabla "ventas_mensual"
6. Ejecute diariamente a las 2 AM
7. Valide que no haya nulos en "total"
'''

requirements = parse_requirements(user_request)
print('Requisitos extraídos:')
print(json.dumps(requirements, indent=2))

## Parte 4: Generador de código ETL

In [ ]:
def generate_etl_pipeline(requirements: dict) -> str:
    """Genera código Python completo del pipeline."""
    prompt = f'''
Genera un pipeline ETL en Python basado en estos requisitos:

{json.dumps(requirements, indent=2)}

El código debe:
- Usar pandas, boto3 (si S3), sqlalchemy (si DB)
- Incluir manejo de errores con try/except
- Logging detallado
- Type hints
- Docstrings
- Función main() ejecutable

Código Python:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

etl_code = generate_etl_pipeline(requirements)
print('Código generado (preview):')
print(etl_code[:500] + '...')

## Parte 5: Validador de código

In [ ]:
def validate_code(code: str) -> dict:
    """Valida sintaxis y seguridad."""
    issues = []
    
    # 1. Validar sintaxis Python
    try:
        ast.parse(code)
    except SyntaxError as e:
        issues.append({'type': 'SYNTAX', 'message': str(e)})
    
    # 2. Detectar patrones inseguros
    dangerous_patterns = ['eval(', 'exec(', 'os.system(', '__import__']
    for pattern in dangerous_patterns:
        if pattern in code:
            issues.append({'type': 'SECURITY', 'message': f'Patrón peligroso detectado: {pattern}'})
    
    # 3. Verificar imports necesarios
    required_imports = ['pandas', 'logging']
    for imp in required_imports:
        if f'import {imp}' not in code:
            issues.append({'type': 'MISSING_IMPORT', 'message': f'Falta import: {imp}'})
    
    return {
        'valid': len(issues) == 0,
        'issues': issues
    }

validation = validate_code(etl_code)
print(f"\nValidación: {'✅ Aprobado' if validation['valid'] else '❌ Con errores'}")
if validation['issues']:
    for issue in validation['issues']:
        print(f"- [{issue['type']}] {issue['message']}")

## Parte 6: Generador de tests

In [ ]:
def generate_tests(code: str, requirements: dict) -> str:
    """Genera tests unitarios con pytest."""
    prompt = f'''
Genera tests unitarios con pytest para este código ETL:

Requisitos:
{json.dumps(requirements, indent=2)}

Código:
{code[:1000]}...

Genera tests para:
1. Lectura de datos (mock de fuente)
2. Transformaciones
3. Validaciones de calidad
4. Escritura (mock de destino)
5. Manejo de errores

Código de tests (pytest):
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

test_code = generate_tests(etl_code, requirements)
print('Tests generados (preview):')
print(test_code[:400] + '...')

## Parte 7: Generador de documentación

In [ ]:
def generate_documentation(requirements: dict, code: str) -> str:
    """Genera README.md completo."""
    prompt = f'''
Genera un README.md completo para este pipeline ETL:

Requisitos:
{json.dumps(requirements, indent=2)}

Incluye:
- Descripción general
- Arquitectura (diagrama ASCII)
- Requisitos (dependencias)
- Configuración
- Cómo ejecutar
- Monitoreo y troubleshooting
- Contacto/owner

Markdown:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.3
    )
    
    return resp.choices[0].message.content.strip()

documentation = generate_documentation(requirements, etl_code)
print('Documentación generada (preview):')
print(documentation[:400] + '...')

## Parte 8: DAG de Airflow

In [ ]:
def generate_airflow_dag(requirements: dict, etl_code: str) -> str:
    """Genera DAG de Airflow."""
    prompt = f'''
Genera un DAG de Airflow para este pipeline:

Requisitos:
{json.dumps(requirements, indent=2)}

El DAG debe:
- Nombre descriptivo
- Schedule según requisitos
- Tasks: extract, validate, transform, load
- Retry logic
- Alertas por email si falla

Código Python del DAG:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.2
    )
    
    return resp.choices[0].message.content.strip().replace('```python','').replace('```','')

dag_code = generate_airflow_dag(requirements, etl_code)
print('DAG de Airflow (preview):')
print(dag_code[:400] + '...')

## Parte 9: Workflow con LangGraph

In [ ]:
from langgraph.graph import StateGraph, END

class PipelineState(TypedDict):
    user_input: str
    requirements: dict
    code: str
    validation: dict
    tests: str
    docs: str
    dag: str
    approved: bool
    errors: Annotated[list, operator.add]

def parse_node(state: PipelineState) -> PipelineState:
    state['requirements'] = parse_requirements(state['user_input'])
    return state

def generate_node(state: PipelineState) -> PipelineState:
    state['code'] = generate_etl_pipeline(state['requirements'])
    return state

def validate_node(state: PipelineState) -> PipelineState:
    state['validation'] = validate_code(state['code'])
    if not state['validation']['valid']:
        state['errors'].extend([i['message'] for i in state['validation']['issues']])
    return state

def test_node(state: PipelineState) -> PipelineState:
    if state['validation']['valid']:
        state['tests'] = generate_tests(state['code'], state['requirements'])
    return state

def document_node(state: PipelineState) -> PipelineState:
    state['docs'] = generate_documentation(state['requirements'], state['code'])
    state['dag'] = generate_airflow_dag(state['requirements'], state['code'])
    return state

def review_node(state: PipelineState) -> PipelineState:
    # En producción: human-in-the-loop
    state['approved'] = state['validation']['valid']
    return state

# Construir grafo
workflow = StateGraph(PipelineState)
workflow.add_node('parse', parse_node)
workflow.add_node('generate', generate_node)
workflow.add_node('validate', validate_node)
workflow.add_node('test', test_node)
workflow.add_node('document', document_node)
workflow.add_node('review', review_node)

workflow.set_entry_point('parse')
workflow.add_edge('parse', 'generate')
workflow.add_edge('generate', 'validate')
workflow.add_edge('validate', 'test')
workflow.add_edge('test', 'document')
workflow.add_edge('document', 'review')
workflow.add_edge('review', END)

app = workflow.compile()

# Ejecutar workflow completo
initial_state = {
    'user_input': user_request,
    'requirements': {},
    'code': '',
    'validation': {},
    'tests': '',
    'docs': '',
    'dag': '',
    'approved': False,
    'errors': []
}

final_state = app.invoke(initial_state)

print('\n🎉 Pipeline generado completamente\n')
print(f"Aprobado: {'✅' if final_state['approved'] else '❌'}")
print(f"Errores: {len(final_state['errors'])}")

## Parte 10: Interfaz Streamlit

In [ ]:
# Guardar como platform_app.py

streamlit_app = '''
import streamlit as st
import json
from pipeline_generator import app, PipelineState  # Importar workflow

st.set_page_config(page_title='GenAI Data Platform', page_icon='🏗️', layout='wide')

st.title('🏗️ Plataforma Self-Service de Pipelines')
st.markdown('Genera pipelines ETL completos usando lenguaje natural')

# Input
user_input = st.text_area(
    'Describe tu pipeline:',
    height=200,
    placeholder='Ej: Necesito procesar datos de ventas desde S3, agregar por mes, y cargar a Redshift...'
)

if st.button('🚀 Generar Pipeline', type='primary'):
    if not user_input:
        st.warning('Por favor describe el pipeline')
    else:
        with st.spinner('Generando pipeline completo...'):
            initial = {
                'user_input': user_input,
                'requirements': {}, 'code': '', 'validation': {},
                'tests': '', 'docs': '', 'dag': '',
                'approved': False, 'errors': []
            }
            
            result = app.invoke(initial)
            
            # Tabs
            tab1, tab2, tab3, tab4, tab5 = st.tabs(['📋 Requisitos', '💻 Código', '🧪 Tests', '📄 Docs', '🛫 DAG'])
            
            with tab1:
                st.json(result['requirements'])
            
            with tab2:
                st.code(result['code'], language='python')
                st.download_button('Descargar código', result['code'], file_name='pipeline.py')
            
            with tab3:
                st.code(result['tests'], language='python')
            
            with tab4:
                st.markdown(result['docs'])
            
            with tab5:
                st.code(result['dag'], language='python')
            
            # Validación
            if result['approved']:
                st.success('✅ Pipeline aprobado y listo para deployment')
            else:
                st.error(f"❌ Pipeline con errores: {result['errors']}")
'''

with open('platform_app.py', 'w') as f:
    f.write(streamlit_app)

print('✅ Plataforma Streamlit guardada en platform_app.py')

## Parte 11: Sistema de deployment

In [ ]:
import subprocess

def deploy_pipeline(code: str, dag: str, tests: str, pipeline_name: str):
    """Despliega pipeline a producción."""
    # 1. Crear estructura de directorios
    base_path = f'./pipelines/{pipeline_name}'
    os.makedirs(base_path, exist_ok=True)
    
    # 2. Guardar archivos
    with open(f'{base_path}/pipeline.py', 'w') as f:
        f.write(code)
    
    with open(f'{base_path}/test_pipeline.py', 'w') as f:
        f.write(tests)
    
    with open(f'{base_path}/dag.py', 'w') as f:
        f.write(dag)
    
    # 3. Ejecutar tests
    test_result = subprocess.run(
        ['pytest', f'{base_path}/test_pipeline.py', '-v'],
        capture_output=True,
        text=True
    )
    
    if test_result.returncode != 0:
        return {
            'success': False,
            'message': 'Tests fallidos',
            'output': test_result.stdout
        }
    
    # 4. Copiar DAG a Airflow
    # airflow_dags_path = '/opt/airflow/dags/'
    # shutil.copy(f'{base_path}/dag.py', f'{airflow_dags_path}/{pipeline_name}.py')
    
    return {
        'success': True,
        'message': f'Pipeline {pipeline_name} desplegado exitosamente',
        'path': base_path
    }

# Deployment
if final_state['approved']:
    deploy_result = deploy_pipeline(
        code=final_state['code'],
        dag=final_state['dag'],
        tests=final_state['tests'],
        pipeline_name=final_state['requirements']['pipeline_name']
    )
    print(deploy_result['message'])

## Parte 12: Monitoreo y observabilidad

In [ ]:
def generate_monitoring_dashboard(pipeline_name: str):
    """Genera dashboard de monitoreo."""
    dashboard_code = f'''
import streamlit as st
import pandas as pd
import plotly.express as px

st.title('📊 Monitoreo: {pipeline_name}')

# Métricas simuladas
col1, col2, col3, col4 = st.columns(4)
col1.metric('Ejecuciones exitosas', '95%', '↑ 2%')
col2.metric('Tiempo promedio', '12.5 min', '↓ 1.2 min')
col3.metric('Registros procesados', '1.2M', '↑ 15K')
col4.metric('Errores', '3', '↓ 5')

# Gráfico de ejecuciones
df_runs = pd.DataFrame({{
    'fecha': pd.date_range('2024-01-01', periods=30),
    'duracion': [10 + i*0.2 for i in range(30)],
    'status': ['success'] * 28 + ['failed'] * 2
}}})

fig = px.line(df_runs, x='fecha', y='duracion', color='status')
st.plotly_chart(fig)

# Logs recientes
st.subheader('Logs recientes')
st.text_area('', value='2024-01-15 02:00 - [INFO] Pipeline iniciado\\n2024-01-15 02:05 - [INFO] 10000 registros procesados', height=200)
'''
    
    return dashboard_code

monitoring_dash = generate_monitoring_dashboard(requirements['pipeline_name'])
print('Dashboard de monitoreo generado')

## Parte 13: Mejoras futuras

### Roadmap

**Fase 2**:
- Multi-agente: equipo de agentes especializados (data engineer, QA, DevOps)
- Cost estimation: predecir costos de infraestructura
- A/B testing: comparar versiones de pipelines
- Auto-scaling: ajustar recursos según carga

**Fase 3**:
- Self-healing: detección y corrección automática de errores
- Optimization: sugerencias de mejora basadas en métricas
- Federated learning: aprender de pipelines de otros equipos
- Natural language alerting: alertas explicadas en lenguaje natural

## Evaluación

**Criterios**:

- ✅ Parsing correcto de requisitos (15%)
- ✅ Generación de código funcional (25%)
- ✅ Validación y seguridad (15%)
- ✅ Tests completos (15%)
- ✅ Documentación clara (10%)
- ✅ Workflow orquestado (10%)
- ✅ Interfaz usable (10%)

## Conclusión

Has construido una plataforma completa de self-service que:

✅ Democratiza el acceso a la ingeniería de datos
✅ Reduce tiempo de desarrollo de días a minutos
✅ Mantiene estándares de calidad y seguridad
✅ Genera documentación automáticamente
✅ Facilita deployment y monitoreo

**¡Felicitaciones por completar el módulo de GenAI!** 🎉